In [504]:
import matplotlib.pyplot as plt
import matplotlib.patches as patch 
import cv2
import numpy as np
import matplotlib.image as mpimg
from sklearn import svm
import math
import sys
import os
from sklearn.metrics import classification_report,accuracy_score

In [537]:
np.seterr(divide='ignore', invalid='ignore')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [505]:
mv = np.zeros((3,3))
mh = np.zeros((3,3))

In [506]:
mv[0][0]=1
mv[0][1]=0
mv[0][2]=-1
mv[1][0]=1
mv[1][1]=0
mv[1][2]=-1
mv[2][0]=1
mv[2][1]=0
mv[2][2]=-1
print(mv)

[[ 1.  0. -1.]
 [ 1.  0. -1.]
 [ 1.  0. -1.]]


In [507]:
mh[0][0]=1
mh[0][1]=1
mh[0][2]=1
mh[1][0]=0
mh[1][1]=0
mh[1][2]=0
mh[2][0]=-1
mh[2][1]=-1
mh[2][2]=-1
print(mh)

[[ 1.  1.  1.]
 [ 0.  0.  0.]
 [-1. -1. -1.]]


In [508]:
def gradient_mag_angle(mv,mh,image):
    for i in range(1,255):
        for j in range(1,255):
            p = image[i-1:i+2,j-1:j+2]
            gv = np.sum(np.dot(p,mv))
            gh = np.sum(np.dot(p,mh))
            m= np.sqrt((gv**2+gh**2))
            #print(mag)
            #print(np.dot(neighbours,mv))
            a = math.degrees(math.atan(gv/gh))
            a = np.nan_to_num(a)
            if gh<0 and gv>=0:
                a = a+180
            if gh<0 and gv<0:
                a = a-180
            if gh == 0 and gv ==0:
                a=0
            if gh==0 and gv>0:
                a=90
            if gh==0 and gv<0:
                a=-90
            #print(a)
            mag[i-1,j-1] = m
            angle[i-1,j-1] = a
    return mag,angle

In [509]:
key_angle = [float(i) for i in range(-180, 180+1, 30)]
#key_angle = float(key_angle)
bins = len(key_angle)
hog = np.zeros((256,bins))

In [513]:
def HOG(mag,angle,patch,image,key_angle,neighbours):
    patch=0
    neighbours = 16
    for i in range (0,256-1,16):
        for j in range (0,256-1,16):
            #print("patch: ",patch)
            row= i+neighbours
            col = j+neighbours
            #print(i,j)
            #print(row,col)
            angle_v = angle[i:row,j:col]
            mag_v = mag[i:row,j:col]
            #print(angle_v.shape,mag_v.shape)
            angle_v = angle_v.flatten()
            mag_v = angle_v.flatten()
            #print(mag_v.shape,angle_v.shape)

            for k in range(256):
                for l in range(len(key_angle)):
                    #print(key_angle[l+1])
                    #print(k,l)

                    if((angle_v[k]>key_angle[l]) and (angle_v[k]<key_angle[l+1])):
                        hog[patch,l] = hog[patch,l]+mag_v[k]*abs(key_angle[l+1]-angle_v[k])/(abs(key_angle[l+1]-angle_v[k])+abs(key_angle[l]-angle_v[k]))
                        hog[patch,l+1] = hog[patch,l+1]+mag_v[k]*abs(key_angle[l]-angle_v[k])/(abs(key_angle[l+1]-angle_v[k])+abs(key_angle[l]-angle_v[k]))

                    else:

                        hog[patch,l] = hog[patch,l] + mag_v[k]

            patch = patch+1 
        return hog

In [514]:
def final_HOG(hog):
    for i in range(hog.shape[1]//2):
        hog[:,i]=hog[:,i]+hog[:,hog.shape[1]-1-i]
        hog = np.delete(hog,[hog.shape[1]-1-i],axis=1)
        #print(hog.shape)
    return hog

In [536]:
dir_images = 'cars_train/'
imgs = os.listdir(dir_images)
feature_dic ={}
neighbours = 16
for imgnm in imgs:
    print(imgnm)
    image = plt.imread(os.path.join(dir_images,imgnm))
    #print(image.shape)
    image = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    image = cv2.resize(image,(256,256))
    mag = np.zeros(image.shape,dtype=np.float64)
    angle = np.zeros(image.shape,dtype=float)
    mag,angle= gradient_mag_angle(mv,mh,image)
    hog = HOG(mag,angle,patch,image,key_angle,neighbours)
    #print(hog.shape)
    row_sums = hog.sum(axis=1)
    new_hog = hog / row_sums[:, np.newaxis]
    new_hog = np.isnan(new_hog)
    feature = final_HOG(new_hog)
    feature = feature.flatten()
    #feature = feature.reshape((-1,1))
    feature = feature.transpose()
    #print(feature.shape)
    feature_dic[imgnm] = feature

car-1081742_640.jpg


/home/surbhi/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in double_scalars
  # Remove the CWD from sys.path while we load stuff.
/home/surbhi/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in true_divide


volga-4856133_640.png
fiat-4298163_640.jpg
sports-car-146873_640.png
plymouth-796441_640.jpg
isolated-3065717_640.png
aston-2118857_640.jpg
car-49278_640.jpg
auto-2179220_640.jpg
classic-car-152118_640.png
car-2158284_640.png
ford-mustang-146580_640.png
car-604019_640.jpg
pontiac-4850448_640.png
car-1957037_640.jpg
chevrolet-3253172_640.png
audi-1890494_640.jpg
ifa-1661767_640.jpg
mercedes-1327610_640.jpg
bmw-918408_640.jpg


In [538]:
dir_images_dog = 'dogs/'
imgs = os.listdir(dir_images_dog)
feature_dog_dic ={}
neighbours = 16
for imgnm in imgs:
    print(imgnm)
    image = plt.imread(os.path.join(dir_images_dog,imgnm))
    #print(image.shape)
    image = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    image = cv2.resize(image,(256,256))
    mag = np.zeros(image.shape,dtype=np.float64)
    angle = np.zeros(image.shape,dtype=float)
    mag,angle= gradient_mag_angle(mv,mh,image)
    hog = HOG(mag,angle,patch,image,key_angle,neighbourss)
    #print(hog.shape)
    row_sums = hog.sum(axis=1)
    new_hog = hog / row_sums[:, np.newaxis]
    new_hog = np.nan_to_num(new_hog)
    feature = final_HOG(new_hog)
    feature = feature.flatten()
    #print(feature.shape)
    #feature = feature.reshape((-1,-1))
    feature = feature.transpose()
    #print(feature.shape)
    feature_dog_dic[imgnm] = feature 

dog-cutout-1-daxi-1408237.jpg
pug-690566_640.jpg
dino-1366698.jpg
3.jpg
puppy-1903313_640.jpg
10.png
6.jpg
puppy.jpg
dog.jpg
dachshund-1519374_640.jpg
11.jpg
cute-puppy-white-background-14660314.jpg
2.jpg
weimaraner-1381186_640.jpg
4.jpg
1jpg
8.jpeg
9.jpg
dog-3277416_640.jpg
7.jpg


In [539]:
hog_car =[]
for key, value in feature_dic.items():
    temp = value
    hog_car.append(temp)
#hog_car = np.asarray(hog_car)
#print(hog_car.shape)

In [540]:
hog_dog = []
for key,value in feature_dog_dic.items():
    temp = value
    hog_dog.append(temp)
#hog_dog = np.asarray(hog_dog)
#print(hog_dog.shape)

In [541]:
X = hog_dog + hog_car
X = np.asarray(X_train)
print(X.shape)

(34, 1792)


In [542]:
classes = [1,0]

In [543]:
label_car = []
for i in range(20):
    label_car.append(classes[0])

In [544]:
label_dog = []
for i in range(20):
    label_dog.append(classes[1])

In [1]:
y = label_dog+label_car
y = np.array(y)
print(y.shape)

NameError: name 'label_dog' is not defined

In [546]:
dir_images_dog = 'dog_test/'
imgs = os.listdir(dir_images_dog)
dog_dic ={}
neighbours = 16
for imgnm in imgs:
    print(imgnm)
    image = plt.imread(os.path.join(dir_images_dog,imgnm))
    #print(image.shape)
    image = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    image = cv2.resize(image,(256,256))
    mag = np.zeros(image.shape,dtype=np.float64)
    angle = np.zeros(image.shape,dtype=float)
    mag,angle= gradient_mag_angle(mv,mh,image)
    hog = HOG(mag,angle,patch,image,key_angle,neighbours)
    #print(hog.shape)
    row_sums = hog.sum(axis=1)
    new_hog = hog / row_sums[:, np.newaxis]
    new_hog = np.nan_to_num(new_hog)
    feature = final_HOG(new_hog)
    
    feature = feature.flatten()
    print(feature.shape)
    #feature = feature.reshape((-1,-1))
    feature = feature.transpose()
    print(feature.shape)
    dog_dic[imgnm] = feature 

3.jpg
(1792,)
(1792,)
puppy.jpg
(1792,)
(1792,)
1.jpg
(1792,)
(1792,)
2.jpg
(1792,)
(1792,)


In [547]:
dir_images = 'car_test/'
imgs = os.listdir(dir_images)
dic ={}
neighbours = 16
for imgnm in imgs:
    print(imgnm)
    image = plt.imread(os.path.join(dir_images,imgnm))
    #print(image.shape)
    image = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    image = cv2.resize(image,(256,256))
    mag = np.zeros(image.shape,dtype=np.float64)
    angle = np.zeros(image.shape,dtype=float)
    mag,angle= gradient_mag_angle(mv,mh,image)
    hog = HOG(mag,angle,patch,image,key_angle,neighbourss)
    #print(hog.shape)
    row_sums = hog.sum(axis=1)
    new_hog = hog / row_sums[:, np.newaxis]
    new_hog = np.nan_to_num(new_hog)
    feature = final_HOG(new_hog)
    feature = feature.flatten()
    #feature = feature.reshape((-1,1))
    feature = feature.transpose()
    #print(feature.shape)
    dic[imgnm] = feature

3136.jpg
car-1300629_640.png
2.jpg
auto-2209439_640.png


In [548]:
car =[]
for key, value in dic.items():
    temp = value
    car.append(temp)

In [549]:
dog = []
for key,value in dog_dic.items():
    temp = value
    dog.append(temp)

In [550]:
X_test = dog+car
X_test = np.asarray(X_test)

In [551]:
X_test.shape

(8, 1792)

In [ ]:
y_test=[]
for i in range(4):
    y_test.append(classes[])
    

In [552]:
y_test = np.asarray(y_test)
print(y_test.shape)

(8,)


In [75]:
classifier = svm.SVC(C=200,kernel='rbf',gamma=0.01,cache_size=8000,probability=False)

In [76]:
model = classifier.fit(X,y)

In [77]:
y_pred = classifier.predict(X_test)

In [78]:
print(y_pred)

[0 0 0 0 0 0 0 0]


In [79]:
print("Accuracy: "+str(accuracy_score(y_test, y_pred)))
print('\n')
print(classification_report(y_test, y_pred))

Accuracy: 0.5


              precision    recall  f1-score   support

           0       0.50      1.00      0.67         4
           1       0.00      0.00      0.00         4

    accuracy                           0.50         8
   macro avg       0.25      0.50      0.33         8
weighted avg       0.25      0.50      0.33         8



/home/surbhi/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
